In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import os
import pickle

import numpy as np
from scipy.special import logit, expit
from sklearn.linear_model import LogisticRegression, Ridge

import utils

In [3]:
PATH_TO_FOLDER_WITH_ORIGIN_DATA = "./chgk"
PATH_TO_FOLDER_WITH_FILTERED_TOURNAMENT_DATA = "./tournaments_data_filtered"
PATH_TO_FOLDER_WITH_SPLITTED_DATA = "./data_splitted_on_train_and_test"

TRAIN_YEAR = 2019
TEST_YEAR = 2020

## Пункт 1: Загрузка и фильтрация данных

In [4]:
FILTER_TOURNAMENTS_DATA = False
SPLIT_DATA_ON_TRAIN_AND_TEST = False

# Фильтрация данных о турнирах
if FILTER_TOURNAMENTS_DATA:
    with open(os.path.join(PATH_TO_FOLDER_WITH_ORIGIN_DATA, "results.pkl"), "rb") as f:
        results = pickle.load(f)
    with open(os.path.join(PATH_TO_FOLDER_WITH_ORIGIN_DATA, "tournaments.pkl"), "rb") as f:
        tournaments = pickle.load(f)
        
    results, tournaments = utils.filter_tournaments_data(results, tournaments, years=(TRAIN_YEAR, TEST_YEAR))
    
    with open(os.path.join(PATH_TO_FOLDER_WITH_SPLITTED_DATA, "results.pkl"), "wb") as f:
        pickle.dump(results, f)
    with open(os.path.join(PATH_TO_FOLDER_WITH_SPLITTED_DATA, "tournaments.pkl"), "wb") as f:
        pickle.dump(tournaments, f)

In [5]:
# Разбивка данных на трейн и тест
if SPLIT_DATA_ON_TRAIN_AND_TEST:
    with open(os.path.join(PATH_TO_FOLDER_WITH_FILTERED_TOURNAMENT_DATA, "results.pkl"), "rb") as f:
        results = pickle.load(f)
    with open(os.path.join(PATH_TO_FOLDER_WITH_FILTERED_TOURNAMENT_DATA, "tournaments.pkl"), "rb") as f:
        tournaments = pickle.load(f)
    
    train_data, test_data = utils.split_data_on_train_and_test(results, tournaments, TRAIN_YEAR, TEST_YEAR)
    
    with open(os.path.join(PATH_TO_FOLDER_WITH_SPLITTED_DATA, "train_data.pkl"), "wb") as f:
        pickle.dump(train_data, f)
    with open(os.path.join(PATH_TO_FOLDER_WITH_SPLITTED_DATA, "test_data.pkl"), "wb") as f:
        pickle.dump(test_data, f)

In [6]:
# Чтение подготовленных данных
with open(os.path.join(PATH_TO_FOLDER_WITH_SPLITTED_DATA, "train_data.pkl"), "rb") as f:
    train_data = pickle.load(f)
with open(os.path.join(PATH_TO_FOLDER_WITH_SPLITTED_DATA, "test_data.pkl"), "rb") as f:
    test_data = pickle.load(f)
with open(os.path.join(PATH_TO_FOLDER_WITH_ORIGIN_DATA, "players.pkl"), "rb") as f:
    players = pickle.load(f)

## Пункт 2: Baseline

**Введем обозначения**:

Пусть имеется множество турниров (*championships*) $\mathbf{C} = \{c_1, \ldots, c_{|\mathbf{C}|} \}$. В турнире $c \in \mathbf{C}$ принимают участие команды $\mathbf{T}^{(c)} = \{t^{(c)}_1, \ldots, t^{(c)}_{|\mathbf{T}^{(c)}|} \},$ где произвольная команда $t$ состоит из игроков $\mathbf{P}_t = \{p_1, \ldots, c_{|\mathbf{P}_t|} \}$. На турнире команды отвечают на вопросы $\mathbf{Q}^{(c)}$.

**Описание базовой модели**:

Предположим, что если команда правильно ответила на вопрос, то ответил и каждый игрок команды. В рамках этого предположения построим модель логистической регрессии оценки вероятности события правильного ответа игроком на поставленный вопрос.

Пусть $i \in \bigcup_{c \in \mathbf{C}} \bigcup_{t \in \mathbf{T}^{(c)}} \mathbf{P}_t $ --- игрок, $s_i$ --- оценка его скилла отвечать на вопросы ЧГК, $q \in \mathbf{Q}^{(c)}$ --- вопрос, $d_q$ --- оценка сложности (*difficulty*) этого вопроса, $\xi_{iq}$ --- бинарный флаг события, в котором игрок $i$ ответил на вопрос $q$. Тогда базовая модель описывается следующим образом:

$$ \mathbf{P} \left[ \xi_{iq}=1 | s_i, d_q \right] = \sigma \left( \alpha + s_i + d_q \right), $$

где $\sigma$ --- сигмоида, $\alpha$ --- константа.

При условии использования такой модели на инференсе для наблюдений тестовой выборки возникнет две проблемы:

1. Прогноз для игроков, отсутствующих в обучающей выборке.
    
    Для этого введем специальный id игрока, равный -1, и заменим на него id таких игроков в обучающей выборке, которые сыграли относительно мало турниров/вопросов и id игроков из тестовой выборки, отсутствующих в обучающей выборке. Таким образом мы избавимся от переобучения на наблюдениях, соответствующих "редким" игрокам, а также это позволит производить инференс для наблюдений с новыми игроками.
    
    
2. Все вопросы в тестовой выборке отсутствуют в обучающей выборке.
        
    Так как инференс на тестовой выборке производится для оценки качества рейтинг-системы, в которой мы, по сути, сравниваем команды в рамках турнира по совокупной силе ее участников, то для тестовой выборки не требуются параметры, соответствующие вопросам. Соответственно, не будем вводить никакую информацию о вопросах в тестовую выборку.

In [7]:
%%time

utils.replace_rare_players_ids_inplace(train_data, q_min=200, replace_id=-1)
utils.replace_unseen_on_train_players_inplace(train_data, test_data, replace_id=-1)

CPU times: user 2min 1s, sys: 45 ms, total: 2min 1s
Wall time: 2min 1s


Конвертация данных в формат разреженных матриц для последующей работы с моделями из *sklearn*.

In [8]:
mapper = utils.get_feature_mapper(train_data)
(
    X_train, 
    y_train, 
    tournament_ids_train,
    team_ids_train,
    player_ids_train,
    tournament_question_nums_train,
) = utils.get_sparse_data_for_sklearn_api(train_data, mapper, regime="train")

(
    X_test, 
    y_test, 
    tournament_ids_test,
    team_ids_test,
    player_ids_test,
    tournament_question_nums_test,
) = utils.get_sparse_data_for_sklearn_api(test_data, mapper, regime="test")

In [9]:
%%time

reg_param_C = 0.1

logistic = LogisticRegression(penalty="l2", C=reg_param_C, solver="saga", tol=0.0001, n_jobs=-1)
logistic.fit(X_train, y_train)

CPU times: user 5min 4s, sys: 248 ms, total: 5min 4s
Wall time: 5min 4s


LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
proba_to_answer_train = logistic.predict_proba(X_train)[:, 1]
proba_to_answer_test = logistic.predict_proba(X_test)[:, 1]

## Пункт 3: Оценка качества рейтинг-системы

Как было указано в предыдущем пункте, для тестирования качества рейтинг-системы нам необходимо отранжировать команды в рамках турниров по совокупной силе участников команды. Для этой цели подойдет ранжирование команд по оценке вероятности события, что команда ответит правильно на абстрактный вопрос, т.е. правильно ответит хотя бы один член команды:

$$ \mathbf{P} \left[ \xi_{t}=1 \right] = 1 - \prod_{i \in t} \left[ 1 - \mathbf{P} \left( \xi_{iq}=1 | s_i \right) \right] = 1 - \prod_{i \in t} \left[ 1 - \sigma \left( \alpha + s_i \right) \right] .$$

In [11]:
true_positions = utils.get_true_positions(test_data)
predicted_positions = utils.get_predicted_positions(
    tournament_ids_test, team_ids_test, proba_to_answer_test
)

kendall_corr, spearman_corr = utils.get_kendall_and_spearman_corr_values(
    true_positions, predicted_positions
)

print(
    f"Оценка качества базовой модели:\n\n"
    f"корреляция Кендалла: {kendall_corr}\n"
    f"корреляция Спирмена: {spearman_corr}"
)

Оценка качества базовой модели:

корреляция Кендалла: 0.6029495740044224
корреляция Спирмена: 0.7572136275463011


## Пункт 4: EM-алгоритм

В базовой модели было сделано предположение о том, что если команда правильно ответила на вопрос, то с вопросом также справился и каждый член команды. Скорее всего, в реальной жизни так не происходит, и для улучшения качества базовой модели попробуем ослабить введенное предположение. Теперь предположим, что в случае, когда команда не ответила на вопрос, то ни один участник не смог правильно ответить, а когда команда справилась --- справился хотя бы один игрок.

Ситуация похожа на случай с presence-only выборкой для оценки вероятности обитания сусликов в областях, где не наблюдался факт обнаружения данных животных. Здесь также воспользуемся EM-схемой, где будем моделировать ненаблюдаемое значение скрытой переменной, определяющей событие, что игрок правильно или неправильно ответил на вопрос при наличии информации о том, как с вопросом справилась команда.

Введем эту бинарную скрытую переменную $z_{iq}$ и рассмотрим ее вероятноcтные свойства:

$ \mathbf{P} \left( z_{iq}=1 | \xi_{tq}=0 \right) = 0 $,

$$ \mathbf{P} \left( z_{iq}=1 | \xi_{tq}=1 \right) = \displaystyle \frac{\mathbf{P} \left( \xi_{tq}=1 | z_{iq}=1 \right) \cdot \mathbf{P} \left( z_{iq}=1 \right)}{\mathbf{P} \left( \xi_{tq}=1 \right)} = \frac{1 \cdot \mathbf{P} \left( z_{iq}=1 \right)}{\mathbf{P} \left( \xi_{tq}=1 \right)} = \frac{\mathbf{P} \left( z_{iq}=1 \right)}{1 - \mathbf{P} \left( \xi_{tq}=0 \right)} = \frac{\mathbf{P} \left( \xi_{iq}=1 \right)}{1 - \prod_{j \in t} \left[ 1 - \mathbf{P} \left( \xi_{jq} = 0 \right) \right]}. $$

Отсюда математическое ожидание скрытой переменной:

$$ \mathbf{E} \left[ z_{iq} \right] =
\begin{equation}
    \left\{ 
      \begin{aligned}
        &0,& \xi_{tq}=0,\\
        &\mathbf{P} \left( z_{iq}=1 | \xi_{tq}=1 \right),& \xi_{tq}=1.\\
      \end{aligned}
     \right.
\end{equation} $$

**EM-схема**:

**_E-шаг_**:

Оцениваем $\mathbf{E} \left[ z_{iq} \right]$ при условии известных параметров скиллов игроков и сложности вопросов.

**_M-шаг_**:

Оцениваем параметры скиллов игроков и сложности вопросов при условии известных ожидаемых значений скрытых переменных:

$$ \mathbf{E} \left[ z_{iq}=1 | s_i, d_q \right] = \sigma \left( \alpha + s_i + d_q \right). $$

In [12]:
n_em_steps = 5

for step in range(n_em_steps):
    # E-шаг
    z_expectation = utils.estimate_latent_vars_expectaion(
        tournament_ids_train,
        team_ids_train,
        player_ids_train,
        tournament_question_nums_train,
        y_train,
        proba_to_answer_train,
    )
    
    # M-шаг
    linear = Ridge(alpha=1/2/reg_param_C, solver="auto", tol=0.0001)
    linear.fit(X_train, logit(z_expectation))
    proba_to_answer_train = expit(linear.predict(X_train))
    proba_to_answer_test = expit(linear.predict(X_test))
    
    # оценка качества
    predicted_positions = utils.get_predicted_positions(
        tournament_ids_test, team_ids_test, proba_to_answer_test
    )
    kendall_corr, spearman_corr = utils.get_kendall_and_spearman_corr_values(
        true_positions, predicted_positions
    )

    print(
        f"EM-шаг {step + 1}:\n"
        f"корреляция Кендалла: {kendall_corr}\n"
        f"корреляция Спирмена: {spearman_corr}\n\n"
    )
    

EM-шаг 1:
корреляция Кендалла: 0.6233189591058687
корреляция Спирмена: 0.7755160885489236


EM-шаг 2:
корреляция Кендалла: 0.6366036562315112
корреляция Спирмена: 0.7899447178534998


EM-шаг 3:
корреляция Кендалла: 0.6359678294586729
корреляция Спирмена: 0.7897426532677517


EM-шаг 4:
корреляция Кендалла: 0.6365040305625046
корреляция Спирмена: 0.790223650031911


EM-шаг 5:
корреляция Кендалла: 0.6366859474217081
корреляция Спирмена: 0.7900781591556572




Алгоритм довольно быстро сошелся, в результате наблюдается незначительное увеличение метрик: корреляции Кендалла и Спирмена выросли примерно на 3-4%.

## Пункт 5: Рейтинг-лист турниров

Оценим списки чемпионатов с самыми сложными/легкими вопросами. Для этого отранжируем чемпионаты по средней оценке сложности вопросов, заданных в рамках чемпионата.

In [13]:
(
    most_difficult_tournaments_names, 
    least_difficult_tournaments_names
) = utils.get_top_n_most_and_least_difficult_tournament_names(linear, mapper, train_data, n=10) 

In [14]:
print("Чемпионаты с наиболее сложными вопросами:\n\n\t", "\n\t".join(most_difficult_tournaments_names), sep="")
print("\n")
print("Чемпионаты с наиболее легкими вопросами:\n\n\t", "\n\t".join(least_difficult_tournaments_names), sep="")

Чемпионаты с наиболее сложными вопросами:

	Чемпионат Санкт-Петербурга. Первая лига
	Воображаемый музей
	Угрюмый Ёрш
	Чемпионат России
	Чемпионат Мира. Этап 2. Группа В
	Первенство правого полушария
	Записки охотника
	Ускользающая сова
	Синхрон высшей лиги Москвы
	Знание – Сила VI


Чемпионаты с наиболее легкими вопросами:

	(а)Синхрон-lite. Лига старта. Эпизод V
	Межфакультетский кубок МГУ. Отбор №4
	(а)Синхрон-lite. Лига старта. Эпизод IX
	(а)Синхрон-lite. Лига старта. Эпизод III
	Открытый чемпионат Белгородской области
	(а)Синхрон-lite. Лига старта. Эпизод VII
	Межфакультетский кубок МГУ. Отбор №3
	Студенческий чемпионат Калининградской области
	Синхрон Лиги Разума
	(а)Синхрон-lite. Лига старта. Эпизод X


Результаты получились довольно логичными. В топе "сложных" турниров находятся чемпионаты России, мира, правого полушария, в "противоположном" списке --- студенческие чемпионаты и Лига старта (видимо, чемпионаты для новичков, гугл не помог их найти описание).